In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)


In [2]:
df = pd.read_csv('../../accepted_2007_to_2018Q4.csv.gz')
df.drop(columns='member_id', inplace=True)


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Feature Engineering

In [3]:
#df = df[df['issue_d'].notna()]

D = (pd.to_datetime(df['issue_d']) - pd.to_datetime(df['earliest_cr_line'])) / np.timedelta64(1, 'Y') 
df['cr_yeargap'] = D.round()

In [4]:
df['issue_d'] = pd.to_datetime(df['issue_d'], format='%b-%Y')

df['issue_month'] = df['issue_d'].dt.month
df['issue_year'] = df['issue_d'].dt.year

df.drop(columns='issue_d', inplace=True)

In [5]:
df.shape

(2260701, 152)

In [6]:
df = df[df['issue_year'] < 2016]
#df = df[df['term'] != ' 60 months']

In [7]:
df.shape

(887440, 152)

In [8]:
df['avg_fico_score'] = (df['fico_range_low']+df['fico_range_high'])/2

In [9]:
df[['home_ownership', 'application_type', 'purpose', 'sub_grade']] =  df[['home_ownership', 'application_type', 'purpose', 'sub_grade']].astype('category')  

df['home_ownership_cat'] = df['home_ownership'].cat.codes 
df['application_type_cat'] = df['application_type'].cat.codes 
df['purpose_cat'] = df['purpose'].cat.codes 
df['sub_grade_cat'] = df['sub_grade'].cat.codes


In [10]:
df.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,cr_yeargap,issue_month,issue_year,avg_fico_score,home_ownership_cat,application_type_cat,purpose_cat,sub_grade_cat
0,68407277,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12.0,2015.0,677.0,1,0,2,13
1,68355089,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.

In [11]:
df.shape

(887440, 157)

## Clean Missing Variables

In [12]:
df['annual_inc_joint'].fillna(df['annual_inc'], inplace=True)
df = df[df['annual_inc_joint'].notna()]

df['dti'].fillna(df['dti_joint'], inplace=True)
df['dti_joint'].fillna(df['dti'], inplace=True)
df = df[df['dti_joint'].notna()]

In [13]:
# only retain numeric data 
df['emp_length'] = df['emp_length'].str.extract('(\d+)', expand=False)

# fill NaN with the most present value in that column
df['emp_length'].fillna(df['emp_length'].mode()[0], inplace=True)

In [14]:
# fill NaN with the mean of that column
mean_ = ['inq_fi', 'inq_last_12m', 'open_acc', 'pub_rec', 'percent_bc_gt_75', 'cr_yeargap']

for col in mean_:
    df[col].fillna(df[col].mean(), inplace=True)

In [15]:
# fill NaN with zero
zero_ = ['pub_rec_bankruptcies', 'total_acc', 'revol_util', 'delinq_amnt', 'delinq_2yrs',\
         'mths_since_last_delinq', 'mths_since_last_major_derog', 'tax_liens']

for col in zero_:
    df[col].fillna(0, inplace=True)

In [16]:
df_clean = df[['term', 'grade','issue_month', 'issue_year', 'purpose', 'annual_inc', 'dti', 'emp_length', 'funded_amnt', 'inq_fi',
       'inq_last_12m', 'installment', 'int_rate', 'loan_status', 'loan_amnt', 'open_acc',
       'percent_bc_gt_75', 'pub_rec', 'pub_rec_bankruptcies', 'tax_liens',
       'total_acc', 'revol_bal', 'delinq_amnt', 'delinq_2yrs',
       'mths_since_last_delinq', 'mths_since_last_major_derog',
       'cr_yeargap', 'avg_fico_score', 'home_ownership_cat',
       'application_type_cat', 'purpose_cat', 'sub_grade_cat', 'home_ownership']].copy()

In [17]:
df_clean.isnull().sum()

term                           0
grade                          0
issue_month                    0
issue_year                     0
purpose                        0
annual_inc                     0
dti                            0
emp_length                     0
funded_amnt                    0
inq_fi                         0
inq_last_12m                   0
installment                    0
int_rate                       0
loan_status                    0
loan_amnt                      0
open_acc                       0
percent_bc_gt_75               0
pub_rec                        0
pub_rec_bankruptcies           0
tax_liens                      0
total_acc                      0
revol_bal                      0
delinq_amnt                    0
delinq_2yrs                    0
mths_since_last_delinq         0
mths_since_last_major_derog    0
cr_yeargap                     0
avg_fico_score                 0
home_ownership_cat             0
application_type_cat           0
purpose_ca

In [18]:
loan_status_dict = {'Fully Paid':1,
                    'Does not meet the credit policy. Status:Fully Paid':1,
                    'Charged Off':0,
                    'Does not meet the credit policy. Status:Charged Off':0,
                    'Late (31-120 days)':0,
                    'Late (16-30 days)':0,
                    'In Grace Period':0,
                    'Current':0,
                    'Default':0}

home_ownership_dict = {'MORTGAGE':'0',
                       'RENT':'1',
                       'OWN':'2',
                       'NONE':'3',
                       'ANY':'3',
                       'OTHER':'3'}

purpose_dict = {'debt_consolidation':'0',
                'credit_card':'1',
                'home_improvement':'2',
                'other':'3',
                'major_purchase':'4',
                'small_business':'5',
                'medical':'6',
                'car':'7',
                'moving':'8',
                'vacation':'9',
                'house':'10',
                'wedding':'11',
                'renewable_energy':'12',
                'educational':'13'
                }

grade_dict = {'A':'0',
             'B':'1',
             'C':'2',
             'D':'3',
             'E':'4',
             'F':'5',
             'G':'6'}


cleanup_dict = {'loan_status':loan_status_dict,
                'home_ownership':home_ownership_dict,
                'purpose':purpose_dict,
                'grade':grade_dict}

df_clean.replace(to_replace=cleanup_dict, value=None, inplace=True)

In [19]:
df_clean.loan_status.value_counts()

1    676286
0    211150
Name: loan_status, dtype: int64

In [20]:
df_clean.home_ownership.value_counts()

0    443591
1    356136
2     87478
3       231
Name: home_ownership, dtype: int64

In [21]:
df_clean.purpose.value_counts()

0     524253
1     206194
2      51834
3      42892
4      17277
5      10378
7       8864
6       8541
8       5415
9       4736
10      3707
11      2347
12       575
13       423
Name: purpose, dtype: int64

In [22]:
df_clean.grade.value_counts()

1    254553
2    245881
0    148203
3    139554
4     70707
5     23049
6      5489
Name: grade, dtype: int64

## Write to csv

In [23]:
df_clean.shape

(887436, 33)

In [24]:
df_clean.to_csv('../../cleaned_accepted_2007_to_2015Q4.csv.gz')